In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import *

In [ ]:
path_to_Gurobi = 'C:/Program Files/Gurobi/win64/bin/gurobi_cl.exe'

### Part a,b -  Minimize number of sensors while covering all pipes

In [ ]:
# Parameters

f = pd.read_csv('Detection_Matrix.csv',header = None)   ## Detection Matrix
f.head()

In [ ]:
## Decision Variables

V = [i for i in range(f.shape[1])]
sv = LpVariable.dicts("s", V, 0, cat='Binary')   # Decision to place sensor at node 'v'

In [ ]:
# Define problem (objective function)

prob = LpProblem("SensorsAssignment", LpMinimize)
prob += lpSum([sv[v] for v in V])

In [ ]:
# Constraints

# Each pipe must have atleast one sensort to detect burst
E = f.shape[0]
for e in range(E):
    prob += lpSum([f.iloc[e,v]*sv[v] for v in V]) >= 1

In [ ]:
# Check solution status
prob.solve(GUROBI_CMD(path=path_to_Gurobi,options=[("MIPGap", 0)]))
print('Solution status: {}'.format(LpStatus[prob.status]))

In [ ]:
a_sensors = []
for v in V:
    if sv[v].varValue == 1:
        a_sensors.append(v)
        print(f'Node {v} has a sensor on it.')

In [ ]:
print(f'Total number of sensors: {prob.objective.value()}')

### Part c,d - Maximize expected number of pipe bursts detected

In [ ]:
### We want to maximize the expected number of pipe bursts detected for different values of number of available sensors ranging from 0 to 20

obj_vals = []
nodes = {}
B = np.arange(21)
for b in B:

    prob = LpProblem("NetworkMonitor", LpMaximize)

    # Probability of pipe burst
    p=0.1
    
    # DECISION VARIABLES

    # "sv" representing whether sensor is placed at node 'v'
    sv = LpVariable.dicts("s", V, 0, cat='Binary')
    #  "y" representing whether a burst is detected at pipe "e" 
    y = LpVariable.dicts("y", [e for e in range(E)], 0, cat='Binary')

    # OBJECTIVE FUNCTION
    prob += lpSum([y[e]*p for e in range(E)])
    
    # CONSTRAINTS
    # 1. Maximum number of sensors that can be placed is 'b'
    prob += lpSum([sv[v] for v in V]) <= b
    # 2. Pipe burst should not be detected if there are no sensors placed to detect it
    for e in range(E):
        prob += lpSum([f.iloc[e,v]*sv[v] for v in V]) >= y[e]
    
    # SOLVE
    prob.solve(GUROBI_CMD(path=path_to_Gurobi,options=[("MIPGap", 0)]))
    
    # Identify nodes where the sensors are placed   
    sensors = []
    for v in V:
        if sv[v].varValue == 1:
            sensors.append(v)
    nodes[b] = sensors
    obj_vals.append(prob.objective.value()) 

    print(f'The expected number of pipes bursts detected for {b} sensors placed is: {prob.objective.value()}')

In [ ]:
# Plotting

plt.plot(B, obj_vals, label='Optimal Value', marker='o', linestyle='-')
plt.title('Optimal Value vs Number of Sensors')
plt.xlabel('Number of Sensors')
plt.ylabel('Expected number of pipe bursts detected')
plt.legend()
plt.grid(True)
plt.show()

### e) Iterative algorithm to maximize expected number of pipe bursts detected

##### We can use a greedy algorithm to iteratively determine the Top 'b' pipes which cover the most sensors

In [ ]:
total_pipe_bursts_detected = []

for b in B:
    f_new = f
    pipe_bursts_detected = 0
    for _ in range(b):
        # Calculate the number of pipes covered by each node
        col_sums = f_new.sum(axis=0)
        
        # Find the node covering the most pipes
        max_col_index = col_sums.idxmax()
        
        # Drop the pipes already covered from the node identified
        f_new = f_new[f_new[max_col_index] != 1]
        # Drop the data related to the node identified to identify the next best node
        f_new = f_new.drop(columns = [max_col_index])
    
    pipe_bursts_detected = f.shape[0] - f_new.shape[0]
    total_pipe_bursts_detected.append(pipe_bursts_detected)
    print(f"The expected pipe bursts detected for {b} sensors are {pipe_bursts_detected}")

In [ ]:
# Plotting

plt.plot(B, total_pipe_bursts_detected, label='Optimal Value', marker='o', linestyle='-')
plt.title('Optimal Value vs Number of Sensors')
plt.xlabel('Number of Sensors')
plt.ylabel('Expected number of pipe bursts detected')
plt.legend()
plt.grid(True)
plt.show()

### Part f,g - Minimize highest criticality of a pipe that is not detected

In [ ]:
# Parameters - criticality matrix

w = pd.read_csv("Criticality.csv", header=None)
w = w[0].values
w

In [ ]:
obj_vals = []
nodes = {}

for b in B:
    # Problem
    prob = LpProblem("NetworkMonitor", LpMinimize)
    
    # DECISION VARIABLES

    # sensor 's' is placed at node 'v' or not
    sv = LpVariable.dicts("s", V, 0, cat='Binary')
    # Burst at pipe 'e' is detected or not
    y = LpVariable.dicts("y", [e for e in range(E)], 0, cat='Binary')
    # 'z' represents the highest criticality score for all the pipes where burst is not detected
    z = LpVariable("z", lowBound = 0, cat='Continuous')

    # OBJECTIVE FUNCTION
    prob += z
    
    # CONSTRAINTS

    #  1. Maximum number of sensors that can be placed is 'b'
    prob += lpSum([sv[v] for v in V]) <= b

    # 2. The value of 'z' should be greater than or equal to all the criticality scores for all the pipes where burst is not detected
    # 3. Pipe burst should not be detected if there are no sensors placed to detect it
    for e in range(E):
        prob += z >= (1-y[e])*w[e]
        prob +=  lpSum([f.iloc[e,v]*sv[v] for v in V]) >= y[e]
    
    # Solve
    prob.solve(GUROBI_CMD(path=path_to_Gurobi,options=[("MIPGap", 0)]))
    
    # Identify nodes where the sensors are placed and 
    sensors = []
    for v in V:
        if sv[v].varValue == 1:
            sensors.append(v)
    nodes[b] = sensors
    obj_vals.append(prob.objective.value())
    print(f'The expected number of pipes bursts detected for {b} sensors placed is: {obj_vals}')